# Introduction

### Get ProteomeScout Phosphorylation Data
- download the phosphorylation zip file from ProteomeScout (https://proteomescout.wustl.edu/compendia/proteomescout_phosphorylation.zip)
- unzip the file 

### Cross Reference with ProteomeScout Phosphorylation Data
-  For each of the resource file, cross reference with ProteomeScout phosphorylation data
    - this step will require the ProteomeScout API
    - the final formatted file only contains data with confirmed phosphorylation site
-  Create a binary substrate/kinase map for each of the filtered resource files


# Cross Referencing with ProteomeScout Phosphorylation Data
### Initializing

In [1]:
# IMPORTS
import XRefProteomeScout
from datetime import date

In [2]:
# version date
ref_version = '2020-02-26'
pscout_version = date.today().strftime('%Y-%m-%d')

# file location
base = '../../'

ref_proteome = base+"Data/Raw/HumanProteome/humanProteome_"+ref_version+".fasta"
pscout_data = base+'Data/Raw/ProteomeScout_'+pscout_version+'/data.tsv'

### Download, Unzip and Save ProteomeScout Phosphorylation Data

In [3]:
XRefProteomeScout.getPScoutData()

### Cross Reference with ProteomeScout Phosphorylation Data

In [ ]:
XRefProteomeScout.XRefProteomeScout(pscout_data, ref_proteome, ref_version)